In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\Kshitij\\Downloads\\F1_CICD\\F1_ML_Ops_CI-CD'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path

In [8]:
from F1_Stint_Prediction.constants import *
from F1_Stint_Prediction.utils.common import read_yaml, create_directories, save_json

In [19]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.Xgboost

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
        )

        return model_evaluation_config

In [16]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score,precision_score,accuracy_score,f1_score,recall_score
from urllib.parse import urlparse
import numpy as np
import joblib

In [22]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_data(self):
        stint_count_model = joblib.load(self.config.model_path + "/stint_count.joblib")
        X_test_stint_count = pd.read_csv(self.config.test_data_path + "/X_test_stint_count_0.csv")
        y_test_stint_count = pd.read_csv(self.config.test_data_path + "/y_test_stint_count_0.csv")
        
        predicted_stint_counts = stint_count_model.predict(X_test_stint_count)
        
        (rmse, mae, r2) = self.eval_metrics_reg(y_test_stint_count, predicted_stint_counts)
        scores = {"rmse": rmse, "mae": mae, "r2": r2}
        save_json(path=Path(self.config.root_dir + "/stint_count.json"), data=scores)
    
        for i in range(1,5):
            model = joblib.load(self.config.model_path + f"/Compound_Stint_{i}.joblib")
            X_test = pd.read_csv(self.config.test_data_path + f"/X_test_compound_{i}.csv")
            y_test = pd.read_csv(self.config.test_data_path + f"/y_test_compound_{i}.csv")
            predicted = model.predict(X_test)
            (accuracy,precision,recall,f1) = self.eval_metrics_class(y_test,predicted)
            scores = {"accuracy": accuracy, "precision": precision, "recall": recall,"f1": f1}
            save_json(path=Path(self.config.root_dir + f"/compound_{i}.json"), data=scores)
            
        for i in range(1,5):
            model = joblib.load(self.config.model_path + f"/Stint_len_{i}.joblib")
            X_test = pd.read_csv(self.config.test_data_path + f"/X_test_stint_len_{i}.csv")
            y_test = pd.read_csv(self.config.test_data_path + f"/y_test_stint_len_{i}.csv")
            predicted = model.predict(X_test)
            (rmse, mae, r2) = self.eval_metrics_reg(y_test,predicted)
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.root_dir + f"/stint_len_{i}.json"), data=scores)
            
            
    def eval_metrics_reg(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def eval_metrics_class(self, actual, pred):
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual, pred, average='weighted', zero_division=0)
        recall = recall_score(actual, pred, average='weighted', zero_division=0)
        f1 = f1_score(actual, pred, average='weighted', zero_division=0)
        return accuracy, precision, recall, f1

In [23]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.eval_data()
except Exception as e:
    raise e

[2025-03-25 12:14:56,277: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-25 12:14:56,285: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-25 12:14:56,291: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-25 12:14:56,292: INFO: common: created directory at: artifacts]
[2025-03-25 12:14:56,294: INFO: common: created directory at: artifacts/model_evaluation]
[2025-03-25 12:14:56,321: INFO: common: json file saved at: artifacts\model_evaluation\stint_count.json]
[2025-03-25 12:14:56,412: INFO: common: json file saved at: artifacts\model_evaluation\compound_1.json]
[2025-03-25 12:14:56,489: INFO: common: json file saved at: artifacts\model_evaluation\compound_2.json]
[2025-03-25 12:14:56,561: INFO: common: json file saved at: artifacts\model_evaluation\compound_3.json]
[2025-03-25 12:14:56,610: INFO: common: json file saved at: artifacts\model_evaluation\compound_4.json]
[2025-03-25 12:14:56,641: INFO: common: json file saved a